In [2]:
# ✅ 0. CSV 로딩 및 전처리 (하루 총 발전량 예측용: scalar target)
import pandas as pd
import numpy as np

# 전체 컬럼 로딩
df = pd.read_csv("df_solar_real_real_final.csv")

In [4]:
import pandas as pd

def eda_summary(df):
    summary = pd.DataFrame({
        '데이터타입': df.dtypes,
        '결측치 수': df.isnull().sum(),
        '결측치 비율(%)': df.isnull().mean() * 100,
        '유니크값 수': df.nunique(),
        '예시값': df.apply(lambda x: x.dropna().unique()[:5])
    })

    # 숫자형만 통계 추가
    num_cols = df.select_dtypes(include='number').columns
    stats = df[num_cols].describe().T
    stats = stats[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
    stats.columns = ['평균', '표준편차', '최솟값', 'Q1', '중앙값', 'Q3', '최댓값']
    
    # 병합
    summary = summary.join(stats, how='left')
    return summary

# 예: df가 분석 대상일 경우
summary_df = eda_summary(df)

# 출력
import pandas as pd
pd.set_option('display.max_columns', None)  # 모든 컬럼 보기
print(summary_df)


             데이터타입    결측치 수  결측치 비율(%)  유니크값 수  \
발전구분        object        0   0.000000      44   
호기           int64        0   0.000000       6   
지점명         object        0   0.000000      18   
설비용량(MW)   float64        0   0.000000      44   
연식(년)      float64        0   0.000000     172   
year         int64        0   0.000000      13   
month        int64        0   0.000000      12   
day          int64        0   0.000000      31   
hour         int64        0   0.000000      24   
weekday      int64        0   0.000000       7   
일사(MJ/m2)  float64  1702875  56.314295     391   
태양고도       float64        0   0.000000  823701   
방위각        float64  1501288  49.647787  823700   
풍속(m/s)    float64     9798   0.324021     178   
풍향(16방위)   float64    12396   0.409937      26   
기온(°C)     float64     8783   0.290455     591   
하늘상태       float64   441519  14.601090       3   
습도(%)      float64     9166   0.303121     140   
강수량(mm)    float64  2560482  84.675468     482   


In [10]:
import pandas as pd

def missing_zero_combined_ratio_by_hour(df):
    result_list = []

    for hour in sorted(df['hour'].dropna().unique()):
        sub_df = df[df['hour'] == hour]
        total_rows = len(sub_df)

        # 비율 계산
        missing_ratio = (sub_df.isnull().sum() / total_rows * 100).round(2)
        zero_ratio = ((sub_df == 0).sum(numeric_only=True) / total_rows * 100).round(2)

        # 모든 컬럼에 대해 비율 계산 후 합산
        all_columns = df.columns
        combined_ratio = (missing_ratio + zero_ratio).reindex(all_columns, fill_value=0).round(2)

        temp_df = pd.DataFrame({
            '시간대': hour,
            '컬럼명': all_columns,
            '결측치 비율(%)': missing_ratio.reindex(all_columns, fill_value=0).values,
            '0값 비율(%)': zero_ratio.reindex(all_columns, fill_value=0).values,
            '결측 + 0 비율(%)': combined_ratio.values
        })

        result_list.append(temp_df)

    result_df = pd.concat(result_list, ignore_index=True)
    return result_df[['시간대', '컬럼명', '결측치 비율(%)', '0값 비율(%)', '결측 + 0 비율(%)']]

# ✅ 사용 예시
result_by_hour = missing_zero_combined_ratio_by_hour(df)

# 보기 좋게 출력 옵션
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(result_by_hour)


     시간대        컬럼명  결측치 비율(%)  0값 비율(%)  결측 + 0 비율(%)
0      1       발전구분       0.00      0.00          0.00
1      1         호기       0.00      0.00          0.00
2      1        지점명       0.00      0.00          0.00
3      1   설비용량(MW)       0.00      0.00          0.00
4      1      연식(년)       0.00      0.26          0.26
5      1       year       0.00      0.00          0.00
6      1      month       0.00      0.00          0.00
7      1        day       0.00      0.00          0.00
8      1       hour       0.00      0.00          0.00
9      1    weekday       0.00     14.27         14.27
10     1  일사(MJ/m2)      93.34      6.66        100.00
11     1       태양고도       0.00    100.00        100.00
12     1        방위각     100.00      0.00        100.00
13     1    풍속(m/s)       0.32     11.11         11.43
14     1   풍향(16방위)       0.37     32.13         32.50
15     1     기온(°C)       0.28      0.29          0.57
16     1       하늘상태      24.03      0.00         24.03
17     1  

In [34]:
import pandas as pd

def find_streaks(df, group_col='지점명', value_col='일사(MJ/m2)', label='일사'):
    # ✅ 일자 생성
    if '일자' not in df.columns:
        if set(['year', 'month', 'day']).issubset(df.columns):
            df['일자'] = pd.to_datetime(df[['year', 'month', 'day']])
        else:
            raise ValueError("일자 컬럼 또는 year/month/day 컬럼이 필요합니다.")
    df['일자'] = pd.to_datetime(df['일자'])  # 혹시라도 str 타입이면

    # ✅ 하루 단위로 결측 또는 0 여부 판단
    daily_check = (
        df.groupby([group_col, '일자'])[value_col]
        .apply(lambda x: x.isna().all() or (x.fillna(0) == 0).all())
        .reset_index(name='is_bad_day')
    )

    # ✅ 연속 결측일 블록 찾기
    results = []
    for name, sub in daily_check.groupby(group_col):
        sub = sub.sort_values('일자').copy()
        sub['block'] = (sub['is_bad_day'] != sub['is_bad_day'].shift()).cumsum()

        for _, block in sub[sub['is_bad_day']].groupby('block'):
            start = block['일자'].iloc[0]
            end = block['일자'].iloc[-1]
            count = len(block)
            if count >= 2:
                results.append({
                    '지점명': name,
                    '결측시작일': start,
                    '결측종료일': end,
                    '연속 결측 시간 수': count,
                    '항목': label
                })

    result_df = pd.DataFrame(results).sort_values(by='연속 결측 시간 수', ascending=False)

    # ✅ 연속 결측 시간 수가 1000 이상인 경우 전체 기간과 함께 출력
    threshold = 1000
    long_missing = result_df[result_df['연속 결측 시간 수'] >= threshold]
    if not long_missing.empty:
        print(f"\n✅ 연속 결측 시간 수 ≥ {threshold}인 지점 요약 ({label}):")
        for name in long_missing['지점명'].unique():
            full_period = df[df[group_col] == name]['일자']
            full_start = full_period.min()
            full_end = full_period.max()
            print(f"\n🔹 지점명: {name}")
            print(f"   - 전체 기간: {full_start.date()} ~ {full_end.date()} ({(full_end - full_start).days + 1}일)")
            for _, row in long_missing[long_missing['지점명'] == name].iterrows():
                print(f"   - {label} 결측 기간: {row['결측시작일'].date()} ~ {row['결측종료일'].date()} "
                      f"({row['연속 결측 시간 수']}시간 연속 결측)")

    return result_df

# ✅ 일사량 결측
sun_result = find_streaks(df, value_col='일사(MJ/m2)', label='일사')

# ✅ 하늘상태 결측
sky_result = find_streaks(df, value_col='하늘상태', label='하늘상태')




✅ 연속 결측 시간 수 ≥ 1000인 지점 요약 (하늘상태):

🔹 지점명: 영월
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   - 하늘상태 결측 기간: 2015-07-11 ~ 2018-07-31 (1116시간 연속 결측)

🔹 지점명: 진주
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   - 하늘상태 결측 기간: 2015-07-13 ~ 2018-07-31 (1115시간 연속 결측)

🔹 지점명: 서귀포
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   - 하늘상태 결측 기간: 2015-07-13 ~ 2018-06-30 (1084시간 연속 결측)


In [ ]:
import pandas as pd

# ✅ 원본 파일과 병합 파일 로딩
df_original = pd.read_csv("df_solar_real_real_final.csv", encoding='utf-8-sig')
df_merged = pd.read_csv("merged_OBS_ASOS_TIM.csv", encoding='utf-8-sig')

if '일자' not in df_original.columns:
    if {'year', 'month', 'day'}.issubset(df_original.columns):
        df_original['일자'] = pd.to_datetime(df_original[['year', 'month', 'day']])
    else:
        raise ValueError("일자 또는 year/month/day 컬럼이 필요합니다.")


# ✅ 일자 형식 정리
df_original['일자'] = pd.to_datetime(df_original['일자'])
df_merged['일자'] = pd.to_datetime(df_merged['일시'])

# 항목별로 가까운 지점 매핑
mapping_sun = {
    '서귀포': '제주',
    '천안': '청주',
    '동해': '강릉',
    '이천': '수원',
    '영월': '원주',
    # 추가 지점...
}

mapping_sky = {
    '진주': '통영',
    '영월': '원주',
    '서귀포': '성산',
    # 추가 지점...
}



In [46]:
def fill_missing_from_merged(df_orig, df_ref, target_col, mapping_dict, threshold=1000, group_col='지점명'):
    df_orig = df_orig.copy()
    df_orig['일자'] = pd.to_datetime(df_orig['일자']).dt.normalize()
    df_ref['일자'] = pd.to_datetime(df_ref['일자']).dt.normalize()

    missing_info = find_streaks(df_orig, group_col=group_col, value_col=target_col, label=target_col)
    target_missing = missing_info[missing_info['연속 결측 시간 수'] >= threshold]

    for _, row in target_missing.iterrows():
        target_station = row['지점명']
        start, end = row['결측시작일'], row['결측종료일']

        if target_station not in mapping_dict:
            print(f"⚠️ 매핑 없음: {target_station}")
            continue

        backup_station = mapping_dict[target_station]
        print(f"🔄 {target_station} → {backup_station} 값으로 {target_col} 결측 채움")

        # ✅ 항목별로 결측 조건 분기
        if target_col == '하늘상태':
            mask_target = (
                (df_orig[group_col] == target_station) &
                (df_orig['일자'] >= start) & (df_orig['일자'] <= end) &
                (df_orig[target_col].isna())
            )
        else:
            mask_target = (
                (df_orig[group_col] == target_station) &
                (df_orig['일자'] >= start) & (df_orig['일자'] <= end) &
                (df_orig[target_col].isna() | (df_orig[target_col] == 0))
            )

        # 백업 값 준비
        backup_data = df_ref[
            (df_ref[group_col] == backup_station) &
            (df_ref['일자'] >= start) & (df_ref['일자'] <= end)
        ].dropna(subset=[target_col])
        
        # ✅ 항목별로 0 필터링 여부 분기
        if target_col != '하늘상태':
            backup_data = backup_data[backup_data[target_col] != 0]
        
        backup_data = backup_data.drop_duplicates(subset='일자')
        backup_series = backup_data.set_index('일자')[target_col]

        # 안전한 reindex로 매핑
        target_dates = df_orig.loc[mask_target, '일자']
        filled_values = backup_series.reindex(target_dates.values)

        # 실제로 채워 넣기
        fill_count = 0
        for idx, val in zip(df_orig[mask_target].index, filled_values):
            if pd.notna(val):
                df_orig.at[idx, target_col] = val
                fill_count += 1

        print(f"✅ 채운 값 개수: {fill_count} / {len(filled_values)}")

    return df_orig




def map_cloud_to_sky(cloud):
    if pd.isna(cloud):
        return None
    try:
        c = float(cloud)
    except:
        return None
    if c <= 2:
        return 1  # 맑음
    elif c <= 7:
        return 3  # 구름 많음
    else:
        return 4  # 흐림

# ✅ df_merged 가공: '하늘상태' 생성
if '전운량(10분위)' in df_merged.columns:
    df_merged = df_merged.rename(columns={'전운량(10분위)': '하늘상태'})
    df_merged['하늘상태'] = df_merged['하늘상태'].apply(map_cloud_to_sky)
else:
    raise ValueError("⚠️ df_merged에 '전운량(10분위)' 컬럼이 없습니다.")

In [47]:
# ✅ 일사량 채우기
df = fill_missing_from_merged(df_original, df_merged, '일사(MJ/m2)', mapping_sun)

# ✅ 하늘상태 채우기
df = fill_missing_from_merged(df, df_merged, '하늘상태', mapping_sky)



✅ 연속 결측 시간 수 ≥ 1000인 지점 요약 (일사(MJ/m2)):

🔹 지점명: 영월
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   - 일사(MJ/m2) 결측 기간: 2013-01-01 ~ 2025-02-28 (4441시간 연속 결측)

🔹 지점명: 서귀포
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   - 일사(MJ/m2) 결측 기간: 2013-01-01 ~ 2024-04-08 (4116시간 연속 결측)

🔹 지점명: 동해
   - 전체 기간: 2018-01-01 ~ 2025-02-28 (2616일)
   - 일사(MJ/m2) 결측 기간: 2018-01-01 ~ 2024-04-17 (2299시간 연속 결측)

🔹 지점명: 천안
   - 전체 기간: 2021-01-01 ~ 2025-02-28 (1520일)
   - 일사(MJ/m2) 결측 기간: 2021-01-01 ~ 2025-02-28 (1520시간 연속 결측)

🔹 지점명: 이천
   - 전체 기간: 2021-02-09 ~ 2025-02-28 (1481일)
   - 일사(MJ/m2) 결측 기간: 2021-02-09 ~ 2025-02-28 (1481시간 연속 결측)
🔄 영월 → 원주 값으로 일사(MJ/m2) 결측 채움
✅ 채운 값 개수: 200160 / 200736
🔄 서귀포 → 제주 값으로 일사(MJ/m2) 결측 채움
✅ 채운 값 개수: 133824 / 133872
🔄 동해 → 강릉 값으로 일사(MJ/m2) 결측 채움
✅ 채운 값 개수: 221496 / 222456
🔄 천안 → 청주 값으로 일사(MJ/m2) 결측 채움
✅ 채운 값 개수: 103584 / 103584
🔄 이천 → 수원 값으로 일사(MJ/m2) 결측 채움
✅ 채운 값 개수: 66408 / 66408

✅ 연속 결측 시간 수 ≥ 1000인 지점 요약 (하늘상태):

🔹 지점명: 영월
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   

In [60]:
# ✅ 보완 완료된 df에 대해 다시 결측 블록 탐색
sun_result = find_streaks(df, value_col='일사(MJ/m2)', label='일사')
sky_result = find_streaks(df, value_col='하늘상태', label='하늘상태')

# ✅ 1000시간 이상 연속 결측만 필터링
sun_remaining = sun_result[sun_result['연속 결측 시간 수'] >= 1000]
sky_remaining = sky_result[sky_result['연속 결측 시간 수'] >= 1000]

# ✅ 결과 요약 출력
print("\n📊 결측치 보완 이후 최종 확인:")

if sun_remaining.empty:
    print("✅ 일사량: 1000시간 이상 연속 결측 지점 없음! 🎉")
else:
    print(f"⚠️ 일사량: {len(sun_remaining)}개 지점에서 여전히 1000시간 이상 연속 결측 있음")

if sky_remaining.empty:
    print("✅ 하늘상태: 1000시간 이상 연속 결측 지점 없음! 🎉")
else:
    print(f"⚠️ 하늘상태: {len(sky_remaining)}개 지점에서 여전히 1000시간 이상 연속 결측 있음")

# ✅ 남은 지점 상세 보기 (선택)
if not sun_remaining.empty:
    print("\n📌 일사량 남은 지점:")
    display(sun_remaining[['지점명', '결측시작일', '결측종료일', '연속 결측 시간 수']])

if not sky_remaining.empty:
    print("\n📌 하늘상태 남은 지점:")
    display(sky_remaining[['지점명', '결측시작일', '결측종료일', '연속 결측 시간 수']])



✅ 연속 결측 시간 수 ≥ 1000인 지점 요약 (하늘상태):

🔹 지점명: 영월
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   - 하늘상태 결측 기간: 2015-07-11 ~ 2018-07-31 (1116시간 연속 결측)

🔹 지점명: 진주
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   - 하늘상태 결측 기간: 2015-07-13 ~ 2018-07-31 (1115시간 연속 결측)

🔹 지점명: 서귀포
   - 전체 기간: 2013-01-01 ~ 2025-02-28 (4442일)
   - 하늘상태 결측 기간: 2015-07-13 ~ 2018-06-30 (1084시간 연속 결측)

📊 결측치 보완 이후 최종 확인:
✅ 일사량: 1000시간 이상 연속 결측 지점 없음! 🎉
⚠️ 하늘상태: 3개 지점에서 여전히 1000시간 이상 연속 결측 있음

📌 하늘상태 남은 지점:


,지점명,결측시작일,결측종료일,연속 결측 시간 수
43,영월,2015-07-11,2018-07-31,1116
69,진주,2015-07-13,2018-07-31,1115
17,서귀포,2015-07-13,2018-06-30,1084


In [61]:
# CSV 파일로 저장 (UTF-8 인코딩)
df.to_csv("solar1.csv", index=False, encoding='utf-8-sig')


In [55]:
def debug_jinju(df, target_col='하늘상태'):
    print("🔍 진주 디버깅 시작")
    
    if '지점명' not in df.columns:
        print("❌ '지점명' 컬럼 없음")
        return
    
    if target_col not in df.columns:
        print(f"❌ '{target_col}' 컬럼 없음")
        return

    # 진주만 필터링
    df_jinju = df[df['지점명'] == '진주'].copy()

    print(f"📌 진주 총 row 수: {len(df_jinju)}")
    
    if len(df_jinju) == 0:
        print("❌ 진주 데이터가 없습니다.")
        return

    # 일자 정보 확인
    if '일자' in df_jinju.columns:
        print(f"📅 진주 데이터 일자 범위: {df_jinju['일자'].min()} ~ {df_jinju['일자'].max()}")
    else:
        print("⚠️ '일자' 컬럼 없음")

    # 결측 확인
    is_na = df_jinju[target_col].isna()
    is_zero = df_jinju[target_col] == 0

    print(f"❓ 결측 수: {is_na.sum()}")
    print(f"❓ 0 포함 여부: {is_zero.sum()}")
    print(f"🔹 고유값: {sorted(df_jinju[target_col].dropna().unique())}")

    # 결측 일자 예시
    if '일자' in df_jinju.columns:
        missing_dates = df_jinju[is_na | is_zero]['일자'].dropna().unique()
        print(f"📆 결측 일자 수: {len(missing_dates)}")
        print(f"📆 예시 (Top 10): {missing_dates[:10]}")

debug_jinju(df, target_col='하늘상태')



🔍 진주 디버깅 시작
📌 진주 총 row 수: 1043112
📅 진주 데이터 일자 범위: 2013-01-01 00:00:00 ~ 2025-02-28 00:00:00
❓ 결측 수: 281613
❓ 0 포함 여부: 0
🔹 고유값: [1.0, 3.0, 4.0]
📆 결측 일자 수: 2232
📆 예시 (Top 10): <DatetimeArray>
['2021-08-09 00:00:00', '2021-08-10 00:00:00', '2021-08-11 00:00:00',
 '2021-08-12 00:00:00', '2021-08-13 00:00:00', '2021-08-14 00:00:00',
 '2021-08-15 00:00:00', '2021-08-16 00:00:00', '2021-08-17 00:00:00',
 '2021-08-20 00:00:00']
Length: 10, dtype: datetime64[ns]


In [59]:
# 진주 결측 시간대
jinju_na = df_original[(df_original['지점명'] == '진주') & (df_original['하늘상태'].isna())][['일자']].copy()
jinju_na['일자'] = pd.to_datetime(jinju_na['일자'])

# 통영에서 해당 시간대 존재 여부 확인
tongyeong = df_original[df_original['지점명'] == '통영'][['일자', '하늘상태']].copy()
tongyeong['일자'] = pd.to_datetime(tongyeong['일자'])

# 진주 결측 중 통영에도 존재하는 날짜 확인
common = pd.merge(jinju_na, tongyeong, on='일자', how='inner')
print(f"✅ 진주 결측 중 통영에도 존재하는 일자 수: {len(common)}")
print(f"📆 예시 (Top 10):\n{common.head(10)}")


✅ 진주 결측 중 통영에도 존재하는 일자 수: 0
📆 예시 (Top 10):
Empty DataFrame
Columns: [일자, 하늘상태]
Index: []


In [23]:
def get_full_period_by_region(df, group_col='지점명', date_col='일자'):
    if date_col not in df.columns:
        if set(['year', 'month', 'day']).issubset(df.columns):
            df[date_col] = pd.to_datetime(df[['year', 'month', 'day']])
        else:
            raise ValueError("일자 또는 year/month/day 컬럼이 필요합니다.")
    df[date_col] = pd.to_datetime(df[date_col])

    period_df = (
        df.groupby(group_col)[date_col]
        .agg(['min', 'max'])
        .reset_index()
        .rename(columns={'min': '시작일', 'max': '종료일'})
    )
    period_df['전체 일 수'] = (period_df['종료일'] - period_df['시작일']).dt.days + 1

    return period_df.sort_values(by='전체 일 수', ascending=False)

full_periods = get_full_period_by_region(df)
print(full_periods)

    지점명                 시작일        종료일  전체 일 수
9    영월 2013-01-01 01:00:00 2025-03-01    4442
12   인천 2013-01-01 01:00:00 2025-03-01    4442
4    부산 2013-01-01 01:00:00 2025-03-01    4442
5   서귀포 2013-01-01 01:00:00 2025-03-01    4442
14   진주 2013-01-01 01:00:00 2025-03-01    4442
2    동해 2018-01-01 01:00:00 2025-03-01    2616
3    목포 2017-01-01 01:00:00 2024-01-01    2556
16   청주 2017-01-01 01:00:00 2023-07-01    2372
7    수원 2017-01-01 01:00:00 2023-07-01    2372
13   제주 2019-12-03 01:00:00 2025-03-01    1915
17   충주 2020-10-01 01:00:00 2025-03-01    1612
15   천안 2021-01-01 01:00:00 2025-03-01    1520
11   이천 2021-02-09 01:00:00 2025-03-01    1481
10   울산 2021-01-01 01:00:00 2024-07-01    1277
0   광양시 2021-01-01 01:00:00 2024-07-01    1277
8   양산시 2023-01-01 01:00:00 2024-01-01     365
6    서산 2023-01-01 01:00:00 2024-01-01     365
1    군산 2024-01-01 01:00:00 2024-10-31     304
